# Chap3 from the book: 'L'apprentissage profond avec Pyhton', François CHOLLET

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import layers
from keras import models
from keras.datasets import boston_housing

In [5]:
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

In [6]:
train_data.shape, test_data.shape

((404, 13), (102, 13))

# Data processing

In [8]:
mean = train_data.mean(axis=0)
train_data -= mean

In [9]:
std = train_data.std(axis=0)
train_data /= std

In [10]:
test_data -= mean
test_data /= std

# Model

In [13]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1], )))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 64)                896       
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.compile(
    loss='mse',
    metrics=['mae'],
    optimizer='rmsprop'
)

In [29]:
# K folds
k = 4
num_val_samples = len(train_data) // k
num_epochs = 200
all_mae_history = []

for i in range(k):
    print("processing fold #", i)
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]

    partial_train_data = np.concatenate([train_data[:i * num_val_samples], train_data[(i+1) * num_val_samples:]], axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples], train_targets[(i+1) * num_val_samples:]], axis=0)

    history = model.fit(
        partial_train_data,
        partial_train_targets,
        epochs=num_epochs,
        validation_data=(val_data, val_targets)
    )
    mae_history = history.history['val_mean_absolute_error']
    all_mae_history.append(mae_history)



10/10 [==============================] - 1s 63ms/step - loss: 0.8517 - mae: 0.5970 - val_loss: 2.1608 - val_mae: 1.1359
Epoch 33/200
10/10 [==============================] - 1s 57ms/step - loss: 0.9813 - mae: 0.6552 - val_loss: 1.2135 - val_mae: 0.7823
Epoch 34/200
10/10 [==============================] - 1s 69ms/step - loss: 0.8440 - mae: 0.5883 - val_loss: 0.9480 - val_mae: 0.6831
Epoch 35/200
10/10 [==============================] - 1s 75ms/step - loss: 1.0264 - mae: 0.6524 - val_loss: 1.3692 - val_mae: 0.8248
Epoch 36/200
10/10 [==============================] - 1s 66ms/step - loss: 0.8951 - mae: 0.6053 - val_loss: 2.0051 - val_mae: 1.0481
Epoch 37/200
10/10 [==============================] - 1s 66ms/step - loss: 0.8983 - mae: 0.6261 - val_loss: 1.1861 - val_mae: 0.7726
Epoch 38/200
10/10 [==============================] - 1s 62ms/step - loss: 0.8352 - mae: 0.5950 - val_loss: 1.1317 - val_mae: 0.7415
Epoch 39/200
10/10 [==============================] - 0s 43ms/step - loss: 0.8012 

KeyError: 'val_mean_absolute_error'

In [26]:
average_mae_history = [np.mean(x[i] for x in all_mae_history) for i in range(num_epochs)]

TypeError: unsupported operand type(s) for /: 'generator' and 'int'

In [ ]:
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Nb epochs')
plt.ylabel('Error (MAE)')
plt.show()

In [ ]:
def smooth_curve(points, factor=0.9):
    smoothed_points = []
    for point in points:
        if smoothed_points:
            previous = smoothed_points[-1]
            smoothed_points.append(previous * factor + point * (1 - factor))
        else:
            smoothed_points.append(point)
    return smoothed_points

In [ ]:
smooth_mae_history = smooth_curve(average_mae_history[10:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Nb epoch')
plt.ylabel('Error (MAE)')

In [ ]:
model.fit(
    train_data,
    train_targets,
    epochs=80,
    batch_size=16,
    verbose=0
)

test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

In [ ]:
test_mae_score